# promptz

In [ ]:
%pip install promptz

In [ ]:
%env OPENAI_API_KEY=<your-api-key>
%env OPENAI_ORGANIZATION_ID=<your-organization-id>

In [3]:

import os
from pprint import pprint
import promptz
from promptz.models.openai import ChatGPT

llm = ChatGPT(
    api_key=os.environ['OPENAI_API_KEY'],
    org_id=os.environ['OPENAI_ORGANIZATION_ID'],
)

promptz.init(llm=llm)

# test session is working
o = promptz.prompt('What is the capital of France')
o

AttributeError: module 'promptz.models' has no attribute 'openai'

In [ ]:
from promptz import prompt

character = 'Batman'
prompt(f'Write a character profile for {character}')

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from promptz import prompt

class Character(BaseModel):
    name: str = Field(..., unique=True, embed=False),
    description: str = Field(
        ..., description='Describe the character in a few sentences')
    age: int = Field(..., min=1, max=120)

characters = prompt(
    'Generate some characters from the Batman universe',
    output=List[Character],
)

In [ ]:
batman = prompt('Generate a character profile for Batman', output=Character)

In [ ]:
descriptions = prompt(
    'Write some 2-3 sentence character descriptions in the style of Alan Moore',
    input={'characters': [c.name for c in characters.objects]},
    output=List[str],
)

characters['description'] = descriptions['output']
characters

In [ ]:
import pandas as pd
from promptz import Prompt

p = Prompt(
    '''
    Generate a list of new characters from the Batman universe.
    Don't use any of the existing characters.
    ''',
    examples=[
        (
            { 'existing_characters': characters['name'][:2].to_list() },
            characters[2:].objects,
        ),
    ],
    output=List[Character],
)

for _ in range(5):
    existing_characters = characters['name'].to_list()
    try:
        cs = prompt(prompt=p, input={'existing_characters': existing_characters})
        characters = pd.concat([characters, cs]).reset_index(drop=True)
    except Exception as e:
        print(e)

characters

In [ ]:
from promptz import store

store(characters)

In [ ]:
from promptz import query

villains = query('they are a villain')
villains

In [ ]:
villains['evil'] = True
store(villains)

In [ ]:
old_and_masked = villains('they wear a mask')[villains['age'] > 30]
old_and_masked

In [ ]:
class StoryIdea(BaseModel):
    title: str
    description: str = None
    characters: List[str] = []

batman = query('Bruce Wayne').first
villains = query('they are a villain').sample(3)

ideas = prompt(
    'Generate some story ideas',
    input={
        'characters': [batman] + villains.objects,
    },
    output=List[StoryIdea],
)

ideas

In [ ]:
from promptz import collection

store(ideas, collection='story_ideas')
collection('story_ideas')